In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
schedule = pd.read_csv("Schedule.csv")
schedule

,G,Date,Web,Away,Opponent,W/L,Pacers,Opp,Notes,Game ID
0,1,20231025,https://www.nba.com/game/was-vs-ind-0022300064...,NaN,WAS,W,143,120,NaN,202310250IND
1,2,20231028,https://www.nba.com/game/ind-vs-cle-0022300091...,@,CLE,W,125,113,NaN,202310280CLE
2,3,20231030,https://www.nba.com/game/chi-vs-ind-0022300102...,NaN,CHI,L,105,112,NaN,202310300IND
3,4,20231101,https://www.nba.com/game/ind-vs-bos-0022300118...,@,BOS,L,104,155,NaN,202311010BOS
4,5,20231103,https://www.nba.com/game/cle-vs-ind-0022300001...,NaN,CLE,W,121,116,In-Season Tournament,202311030IND
...,...,...,...,...,...,...,...,...,...,...
62,63,20240305,https://www.nba.com/game/ind-vs-dal-0022300892...,@,DAL,W,137,120,NaN,202403050DAL
63,64,20240307,https://www.nba.com/game/min-vs-ind-0022300903...,NaN,MIN,L,111,113,NaN,202403070IND
64,65,20240310,https://www.nba.com/game/ind-vs-orl-0022300928...,@,ORL,W,111,97,NaN,202403100ORL
65,66,20240312,https://www.nba.com/game/ind-vs-okc-0022300941...,@,OKC,W,121,111,NaN,202403120OKC


In [3]:
schedule.rename(columns={'Web':'url'}, inplace =True)

In [11]:
schedule['url'] = schedule['url'].str.replace('#box-score$', '?type=traditional', regex=True)
schedule

,G,Date,url,Away,Opponent,W/L,Pacers,Opp,Notes,Game ID
0,1,20231025,https://www.nba.com/game/was-vs-ind-0022300064...,NaN,WAS,W,143,120,NaN,202310250IND
1,2,20231028,https://www.nba.com/game/ind-vs-cle-0022300091...,@,CLE,W,125,113,NaN,202310280CLE
2,3,20231030,https://www.nba.com/game/chi-vs-ind-0022300102...,NaN,CHI,L,105,112,NaN,202310300IND
3,4,20231101,https://www.nba.com/game/ind-vs-bos-0022300118...,@,BOS,L,104,155,NaN,202311010BOS
4,5,20231103,https://www.nba.com/game/cle-vs-ind-0022300001...,NaN,CLE,W,121,116,In-Season Tournament,202311030IND
...,...,...,...,...,...,...,...,...,...,...
62,63,20240305,https://www.nba.com/game/ind-vs-dal-0022300892...,@,DAL,W,137,120,NaN,202403050DAL
63,64,20240307,https://www.nba.com/game/min-vs-ind-0022300903...,NaN,MIN,L,111,113,NaN,202403070IND
64,65,20240310,https://www.nba.com/game/ind-vs-orl-0022300928...,@,ORL,W,111,97,NaN,202403100ORL
65,66,20240312,https://www.nba.com/game/ind-vs-okc-0022300941...,@,OKC,W,121,111,NaN,202403120OKC


In [6]:
# Function to scrape data from a given URL
def scrape_data(url):
    # chrome driver to scrape dynamic webpages
    driver = webdriver.Chrome()
    driver.get(url)

    # beautiful soup to parse it
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all tables on the page
    tables = soup.find_all('table')

    # Select the second table (indexing starts from 0)
    t = tables[1]

    # Extract headers and data from the table
    headers = [i.text.strip() for i in t.find_all('th')]
    data = [[i.text.strip() for i in b.find_all('td')] for b in t.find_all('tr')]

    # Remove empty rows
    data = [row for row in data if len(row) > 1]

    # Create DataFrame
    df = pd.DataFrame(data, columns=headers)

    return df

# Initialize an empty list to store DataFrames
combined_dfs = []

# Iterate through the schedule DataFrame and scrape data for rows where "Away" is NaN
for index, row in schedule.iterrows():
    if pd.isna(row['Away']):
        game_id = row['Game ID']
        url = row['url']
        print(f"Scraping data for Game ID: {game_id}")

        # Scrape data from the URL
        df = scrape_data(url)

        # Add Game ID to the DataFrame
        df['Game ID'] = game_id

        # Append the scraped DataFrame to the list
        combined_dfs.append(df)

# Concatenate all scraped DataFrames into a single DataFrame
box_home = pd.concat(combined_dfs, ignore_index=True)

# Display the combined DataFrame
print(box_home)

Scraping data for Game ID: 202310250IND
Scraping data for Game ID: 202310300IND
Scraping data for Game ID: 202311030IND
Scraping data for Game ID: 202311040IND
Scraping data for Game ID: 202311060IND
Scraping data for Game ID: 202311080IND
Scraping data for Game ID: 202311090IND
Scraping data for Game ID: 202311190IND
Scraping data for Game ID: 202311220IND
Scraping data for Game ID: 202311240IND
Scraping data for Game ID: 202311270IND
Scraping data for Game ID: 202312040IND
Scraping data for Game ID: 202312180IND
Scraping data for Game ID: 202312200IND
Scraping data for Game ID: 202312230IND
Scraping data for Game ID: 202312300IND
Scraping data for Game ID: 202401030IND
Scraping data for Game ID: 202401050IND
Scraping data for Game ID: 202401060IND
Scraping data for Game ID: 202401080IND
Scraping data for Game ID: 202401100IND
Scraping data for Game ID: 202401230IND
Scraping data for Game ID: 202401250IND
Scraping data for Game ID: 202401260IND
Scraping data for Game ID: 202401280IND


In [14]:
# Scrape away games:
# Function to scrape data from a given URL
def scrape_data(url):
    # chrome driver to scrape dynamic webpages
    driver = webdriver.Chrome()
    driver.get(url)

    # beautiful soup to parse it
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all tables on the page
    tables = soup.find_all('table')

    # Select the second table (indexing starts from 0)
    t = tables[0]

    # Extract headers and data from the table
    headers = [i.text.strip() for i in t.find_all('th')]
    data = [[i.text.strip() for i in b.find_all('td')] for b in t.find_all('tr')]

    # Remove empty rows
    data = [row for row in data if len(row) > 1]

    # Create DataFrame
    df = pd.DataFrame(data, columns=headers)

    return df

# Initialize an empty list to store DataFrames
box_away_dfs = []

# Iterate through the schedule DataFrame and scrape data for rows where "Away" is NaN
for index, row in schedule.iterrows():
    if not pd.isna(row['Away']):
        game_id = row['Game ID']
        url = row['url']
        print(f"Scraping data for Game ID: {game_id}")

        # Scrape data from the URL
        df = scrape_data(url)

        # Add Game ID to the DataFrame
        df['Game ID'] = game_id

        # Append the scraped DataFrame to the list
        box_away_dfs.append(df)

# Concatenate all scraped DataFrames into a single DataFrame
box_away = pd.concat(box_away_dfs, ignore_index=True)

Scraping data for Game ID: 202310280CLE
Scraping data for Game ID: 202311010BOS
Scraping data for Game ID: 202311120PHI
Scraping data for Game ID: 202311140PHI
Scraping data for Game ID: 202311210ATL
Scraping data for Game ID: 202311300MIA
Scraping data for Game ID: 202312020MIA
Scraping data for Game ID: 202312070MIL
Scraping data for Game ID: 202312110DET
Scraping data for Game ID: 202312130MIL
Scraping data for Game ID: 202312150WAS
Scraping data for Game ID: 202312160MIN
Scraping data for Game ID: 202312210MEM
Scraping data for Game ID: 202312260HOU
Scraping data for Game ID: 202312280CHI
Scraping data for Game ID: 202401010MIL
Scraping data for Game ID: 202401120ATL
Scraping data for Game ID: 202401140DEN
Scraping data for Game ID: 202401150UTA
Scraping data for Game ID: 202401180SAC
Scraping data for Game ID: 202401190POR
Scraping data for Game ID: 202401210PHO
Scraping data for Game ID: 202401300BOS
Scraping data for Game ID: 202402010NYK
Scraping data for Game ID: 202402040CHO


In [15]:
combinedbox

,PLAYER,MIN,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,+/-,Game ID
0,Bennedict MathurinB. MathurinF,17:04,5,10,50.0,1,3,33.3,2,3,...,0,0,1,0,0,0,0,13,-11,202310280CLE
1,Obi ToppinO. ToppinF,18:00,2,5,40.0,0,1,0.0,0,0,...,2,4,0,0,1,1,1,4,-6,202310280CLE
2,Myles TurnerM. TurnerC,29:25,7,14,50.0,1,5,20.0,5,5,...,11,12,1,0,2,2,5,20,-5,202310280CLE
3,Bruce BrownB. BrownG,32:11,3,8,37.5,0,1,0.0,0,0,...,4,4,2,1,0,0,3,6,0,202310280CLE
4,Tyrese HaliburtonT. HaliburtonG,32:01,7,16,43.8,4,7,57.1,3,3,...,7,8,13,0,0,1,2,21,-1,202310280CLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,Jarace WalkerJ. Walker,6:51,1,2,50.0,1,2,50.0,0,0,...,1,2,0,0,0,0,0,3,5,202403120OKC
478,Kendall BrownK. Brown,DNP - Coach's Decision,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,202403120OKC
479,Isaiah JacksonI. Jackson,DNP - Coach's Decision,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,202403120OKC
480,James JohnsonJ. Johnson,DND - Injury/Illness,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,202403120OKC


In [16]:
combined_box = pd.concat([box_home, box_away], ignore_index=True)
combined_box.to_csv('combined_box.csv', index=False)

In [17]:
combined_box

,PLAYER,MIN,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,+/-,Game ID
0,Bennedict MathurinB. MathurinF,25:50,6,15,40.0,2,6,33.3,4,4,...,4,4,5,2,0,2,1,18,14,202310250IND
1,Obi ToppinO. ToppinF,19:14,4,9,44.4,2,5,40.0,1,1,...,3,4,1,1,0,0,0,11,7,202310250IND
2,Myles TurnerM. TurnerC,23:20,5,8,62.5,0,2,0.0,1,2,...,8,8,1,0,3,2,4,11,14,202310250IND
3,Bruce BrownB. BrownG,27:12,8,11,72.7,6,8,75.0,2,2,...,2,3,1,1,0,1,3,24,15,202310250IND
4,Tyrese HaliburtonT. HaliburtonG,26:54,9,16,56.3,2,6,33.3,0,0,...,3,3,11,1,2,3,1,20,6,202310250IND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Jarace WalkerJ. Walker,6:51,1,2,50.0,1,2,50.0,0,0,...,1,2,0,0,0,0,0,3,5,202403120OKC
996,Kendall BrownK. Brown,DNP - Coach's Decision,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,202403120OKC
997,Isaiah JacksonI. Jackson,DNP - Coach's Decision,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,202403120OKC
998,James JohnsonJ. Johnson,DND - Injury/Illness,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,202403120OKC
